<a href="https://colab.research.google.com/github/mariatabi/cv-tasks/blob/main/try6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops

In [14]:
from scipy.ndimage import morphology
from skimage.filters import threshold_triangle
from skimage.filters import threshold_otsu
 
def lakes(image):
    B = ~image
    BB = np.ones((B.shape[0] + 2, B.shape[1] + 2))
    BB[1:-1, 1:-1] = B
    return np.max(label(BB)) - 1
 
def has_vline(image):
    lines = np.sum(image, 0) // image.shape[0]
    return 1 in lines

def has_hline(image):
    lines = np.sum(image, 1) // image.shape[1]
    return 1 in lines

def has_bay(image):
    b = ~image
    bb = np.zeros((b.shape[0] + 1, b.shape[1])).astype("uint8")
    bb[:-1, :] = b
    return lakes(~bb) - 1
 
def count_bays(image):
    holes = ~image.copy()
    return np.max(label(holes))
 
def recognize(region):
    bays = count_bays(region.image)
    lc = lakes(region.image)
    circ = region.perimeter ** 2 / region.area
    if lc == 2:
        if has_vline(region.image) and bays < 5:
            return "B"
        return "8"

    if lc == 1:
        if has_bay(region.image) > 0:
            return "A"

        if has_vline(region.image):
            circ = region.perimeter ** 2 / region.area
            if bays > 3:
                return "0"
            if (circ > 58):
                return "D"
            return"P"
        return "0"
            
    if lc == 0:
        if has_vline(region.image) and (bays == 0 or bays == 3):
            if np.all(region.image == 1):
                return "-"
            return "1"
        
        if bays == 2:
            return "/"

        if bays > 3:
            circ = region.perimeter ** 2 / region.area
            if bays == 5:
                if has_hline(region.image):
                    return '*'
                return 'W'
            else: 
                if bays == 4 and circ > 40:
                    return "X"
                else:
                    return "*"
    return None
 
def show_symbols(SYM, regions):
    a, b, c = 1, 9, 1
    for region in regions:
        symbol = recognize(region)
        if c == 10:
            c = 1
            a += 1

        if a == 10:
            a = 1
            c += 1

        if symbol == SYM:
            case = str(a)+str(b)+str(c)
            c += 1
 
image = plt.imread("symbols.png")
image = np.sum(image, 2)
image[image > 0] = 1
 
labeled = label(image)
print("labeled: ", np.max(labeled))
 
regions = regionprops(labeled)
d = {}
for region in regions:
    symbol = recognize(region)
    if symbol not in d:
        d[symbol] = 1
    else:
        d[symbol] += 1
 
for key in d.keys():
    print("symbol: ", key)
    show_symbols(key, regions)
    procent = d.get(key) / sum(d.values()) * 100
    print(procent, "%")
    
    
print(d)
print("all: ", sum(d.values()))

plt.show()
#plt.figure()
#plt.subplot(121)
#plt.imshow(image)
#plt.subplot(122)
#plt.imshow(labeled)


labeled:  400
symbol:  D
7.75 %
symbol:  X
5.75 %
symbol:  /
8.75 %
symbol:  *
10.25 %
symbol:  1
10.0 %
symbol:  A
8.75 %
symbol:  P
9.25 %
symbol:  8
8.25 %
symbol:  -
7.75 %
symbol:  B
9.5 %
symbol:  W
6.5 %
symbol:  0
7.5 %
{'D': 31, 'X': 23, '/': 35, '*': 41, '1': 40, 'A': 35, 'P': 37, '8': 33, '-': 31, 'B': 38, 'W': 26, '0': 30}
all:  400
